In [ ]:
# Работа с массивами данных
import numpy as np

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential, Model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input, concatenate
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt

# Метрики для расчета ошибок
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Загрузка датасета
df = pd.read_csv('/content/drive/MyDrive/SivilioBel_all_info_ver1.csv', index_col=0)

# Вывод первых нескольких строк датасета для проверки
print(df.head(1))
print(df.shape)

  Тип события Вакансия                      Компания        Кандидат  \
№                                                                      
1       Оффер  ML Lead  PersonalInvest - Indata Labs  Бакаев Евгений   

  Дата события                                 Примечание события  \
№                                                                   
1   01.04.2024  01.04 - на рассмотрение 07.04 - интервью Бакае...   

                          Скилы для вакансии Основной навык для вакансии  \
№                                                                          
1  AWS, Azure, ETL, Machine Learning, python            Machine Learning   

  Зарплата для вакансии Уровень для вакансии  ... Зарплата кандидата  \
№                                             ...                      
1    8000-12000 в месяц              Head of  ...                NaN   

  Опыт работы кандидата График кандидата Регион кандидата Место кандидата  \
№                                                   

In [ ]:
# Замена концов строк на пробелы, удаление символа с кодом 0xA0
# обрезка краевых пробелов, приведение к нижнему регистру

def purify(x):
    if isinstance(x, str):                # Если значение - строка:
        x = x.replace('\n', ' ').replace('\xa0', '').strip().lower()
    return x

In [ ]:
# Настройка номеров столбцов

COL_IVENT_TYPE     = df.columns.get_loc('Тип события')
COL_VACANCY     = df.columns.get_loc('Вакансия')
COL_PERSON     = df.columns.get_loc('Кандидат')
COL_SKILLS     = df.columns.get_loc('Скилы для вакансии')
COL_BASE_SKILL     = df.columns.get_loc('Основной навык для вакансии')
COL_VAC_SALARY      = df.columns.get_loc('Зарплата для вакансии')
COL_LEVEL_VACANCY     = df.columns.get_loc('Уровень для вакансии') #Уровень, требуемый для вакансии
# COL_POS_SEEK    = df.columns.get_loc('Ищет работу на должность:')
COL_SALARY      = df.columns.get_loc('Зарплата кандидата')
COL_POS_PREV    = df.columns.get_loc('Профессия кандидата')
COL_CITY        = df.columns.get_loc('Регион кандидата')
COL_EMPL        = df.columns.get_loc('Место кандидата') #Удаленная/офис и пр.
COL_SCHED       = df.columns.get_loc('График кандидата')
COL_EXP         = df.columns.get_loc('Опыт работы кандидата')
COL_ENG         = df.columns.get_loc('Английский кандидата')
# COL_EDU         = df.columns.get_loc('Образование и ВУЗ')
COL_UPDATED     = df.columns.get_loc('Ссылка на резюме')

# Вывод первых нескольких строк датасета для проверки
print(df.head(5))
print(df.shape)

        Тип события                      Вакансия  \
№                                                   
1             Оффер                       ML Lead   
2      Трудоустроен                    QA/Support   
3      Трудоустроен  digital marketing specialist   
4             Отказ                        DevOps   
5  Тестовое задание            Frontend Developer   

                       Компания         Кандидат Дата события  \
№                                                               
1  PersonalInvest - Indata Labs   Бакаев Евгений   01.04.2024   
2                 Immersive Fox   Лобань Дмитрий   15.04.2024   
3                 Immersive Fox     Куца Татьяна   10.05.2024   
4                    SolbegSoft  Мельничук Павел   06.05.2024   
5                    SolbegSoft       Попов Олег   03.05.2024   

                                  Примечание события  \
№                                                      
1  01.04 - на рассмотрение 07.04 - интервью Бакае...   
2  1

In [ ]:
from tabulate import tabulate

n = 2  # Индекс в таблице резюме
data = []
for i in range(df.columns.shape[0]):  # Вывод значения каждого столбца
    data.append([f'\033[91m{df.columns[i]}:\033[0m', df.values[n][i]])

print(tabulate(data, tablefmt="grid"))

+------------------------------+----------------------------------------------------------------------+
| Тип события:                 | Трудоустроен                                                         |
+------------------------------+----------------------------------------------------------------------+
| Вакансия:                    | digital marketing specialist                                         |
+------------------------------+----------------------------------------------------------------------+
| Компания:                    | Immersive Fox                                                        |
+------------------------------+----------------------------------------------------------------------+
| Кандидат:                    | Куца Татьяна                                                         |
+------------------------------+----------------------------------------------------------------------+
| Дата события:                | 10.05.2024                     

In [ ]:
# Проведем анализ датасета
df.shape # Кол-во строк и столбцов в датасете
df.count() # Кол-во непустых записей в каждом из столбцов
df.dtypes # Тип данных в каждом из столбцов
df.info() # Саммари по датасету

<class 'pandas.core.frame.DataFrame'>
Index: 6212 entries, 1 to 6212
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Тип события                  6212 non-null   object 
 1   Вакансия                     6212 non-null   object 
 2   Компания                     6212 non-null   object 
 3   Кандидат                     6212 non-null   object 
 4   Дата события                 6179 non-null   object 
 5   Примечание события           6153 non-null   object 
 6   Скилы для вакансии           6157 non-null   object 
 7   Основной навык для вакансии  5338 non-null   object 
 8   Зарплата для вакансии        5363 non-null   object 
 9   Уровень для вакансии         6060 non-null   object 
 10  Английский для вакансии      5768 non-null   object 
 11  Место для вакансии           6119 non-null   object 
 12  График для вакансии          6123 non-null   object 
 13  Уровень кандидата      

In [ ]:
df.drop(['Примечание кандидата', 'Ссылка на резюме'], axis=1, inplace=True)

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 6212 entries, 1 to 6212
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Тип события                  6212 non-null   object 
 1   Вакансия                     6212 non-null   object 
 2   Компания                     6212 non-null   object 
 3   Кандидат                     6212 non-null   object 
 4   Дата события                 6179 non-null   object 
 5   Примечание события           6153 non-null   object 
 6   Скилы для вакансии           6157 non-null   object 
 7   Основной навык для вакансии  5338 non-null   object 
 8   Зарплата для вакансии        5363 non-null   object 
 9   Уровень для вакансии         6060 non-null   object 
 10  Английский для вакансии      5768 non-null   object 
 11  Место для вакансии           6119 non-null   object 
 12  График для вакансии          6123 non-null   object 
 13  Уровень кандидата      